In [1]:
import openai
from pathlib import Path
import json
import llm2geneset
import time
import pandas as pd
import numpy as np
from rouge_score import rouge_scorer
from statsmodels.stats.multitest import multipletests

aclient = openai.AsyncClient()
client = openai.Client()

In [2]:
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2'], use_stemmer=True)

In [3]:
def flatten_list_of_lists(lst):
    return [item for sublist in lst for item in sublist]

def unflatten_list_of_lists(flat_list, original_structure):
    result = []
    index = 0
    for sublist in original_structure:
        length = len(sublist)
        result.append(flat_list[index:index + length] if length > 0 else [])
        index += length
    return result

In [4]:
models = ["gpt-3.5-turbo-0125","gpt-4o-2024-05-13"]
lib_names = ["KEGG_2021_Human",
             "Reactome_2022", 
             "WikiPathway_2023_Human"]

output = []
for model in models:
    print(model)
    for lib_name in lib_names:
        print(lib_name)
        with open("libs_human/" + model + "/" + lib_name + ".json") as f:
            gen_res = json.load(f)
        
        test_sets = gen_res["curated_genesets"]
        test_descr = gen_res["descr"]
        test_descr_cleaned = gen_res["descr_cleaned"]

        gt_embs = llm2geneset.get_embeddings(client, test_descr_cleaned)

        for method in ["llm2geneset", "GSAI"]:
            if method == "GSAI":
                # use GSAI to generate geneset name
                gsai_res = await llm2geneset.gsai(aclient, test_sets, model=model, n_retry=3)
                # use lists of size 1 for names.
                names = [[i['name']]for i in gsai_res]
                in_toks = [i["in_toks"] for i in gsai_res]
                out_toks = [i["out_toks"] for i in gsai_res]
            elif method == "llm2geneset":
                llm2geneset_res = await llm2geneset.gs_proposal(aclient, test_sets, model=model)
                # Extract multiple names meeting significance.
                def res2name(res):
                    df = res["ora_results"]
                    _, pvals_corrected, _, _ = multipletests(df["p_val"], method='bonferroni')
                    df["p_adj"] = pvals_corrected
                    df = df[df["p_adj"] < 0.01]
                    if df.shape[0] == 0:
                        return ["None found."]
                    else: 
                        return df["bio_process"].to_list()            
                names = list(map(res2name, llm2geneset_res))
                in_toks = [i["tot_in_toks"] for i in llm2geneset_res]
                out_toks = [i["tot_out_toks"] for i in llm2geneset_res]

            # Since there can be multiple names per gene set
            # compute cosine similarity for each one. 
            flat_names = flatten_list_of_lists(names)
            flat_name_embs = llm2geneset.get_embeddings(client, flat_names)
            name_emb = unflatten_list_of_lists(flat_name_embs,names)

            # Get best score for significant gene sets and report. 
            for i, ref in enumerate(test_descr_cleaned):
                scores = [scorer.score(ref, n) for n in names[i]] 
                rouge1 = [s['rouge1'].recall for s in scores]
                rouge2 = [s['rouge2'].recall for s in scores]

                csim = [np.dot(gt_embs[i], emb) for emb in name_emb[i]]
                    
                x={"model":model,
                   "library":lib_name,
                   "gt_name":test_descr[i],
                   "gt_name_clean":ref,
                   "name":", ".join(names[i]),
                   "ROUGE1":max(rouge1),
                   "ROUGE2":max(rouge2),  
                   "csim":max(csim),
                   "method": method,
                   "in_toks":in_toks[i],
                   "out_toks": out_toks[i]                   
                  }    
            
                output.append(x)

gpt-3.5-turbo-0125
KEGG_2021_Human


 13%|███████████████▊                                                                                                           | 41/320 [00:07<00:20, 13.75it/s]

retrying
List 5 biological pathways, biological processes, or cellular components that contain the following genes """KL,MIOX,UGT2B11,UGT2B10,UGT2A1,UGT2B15,UGT1A1,UGT2B17,AKR1A1,ALDH3A2,ALDH7A1,UGDH,RGN,UGT2B4,ALDH9A1,UGT2B7,UGT1A10,UGT2B28,ALDH1B1,ALDH2,UGT1A6,UGT1A5,UGT1A4,UGT2A3,UGT1A3,UGT2A2,GUSB,UGT1A9,UGT1A8,UGT1A7""" with high confidence. Be as specific as possible. Use the following JSON schema:
```json
{
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "p": {
                "type": "string",
            },
        },
        "required": ["p"]
    }
}
```
Example output will look like the following:
```json
[{"p":"bp_or_pathway1"},
 {"p":"bp_or_pathway2"},
 {"p":"bp_or_pathway3"},
 {"p":"bp_or_pathway4"},
```
The element `p` designates a pathway, biological process or cellular component. Place the output in a JSON code block. Do not add any comments in the JSON code block.

No pathways returned.
```json
[]
```


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 320/320 [00:18<00:00, 17.77it/s]


Reactome_2022


...

 97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎   | 777/801 [00:13<00:01, 23.05it/s]

retrying
conf is none
Write a critical analysis of the biological processes performed by this system of interacting proteins.

Base your analysis on prior knowledge available in your training data.  After completing your analysis, propose a brief and
detailed name for the most prominent biological process performed by the system.

After completing your analysis, please also assign a confidence score to the process name you selected.  This score should
follow the name in parentheses and range from 0.00 to 1.00. A score of 0.00 indicates the lowest confidence, while 1.00 reflects
the highest confidence. This score helps gauge how accurately the chosen name represents the functions and activities within
the system of interacting proteins. When determining your score, consider the proportion of genes in the protein system that
participate in the identified biological process. For instance, if you select "Ribosome biogenesis" as the process name but
only a few genes in the system contribute

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 799/801 [00:18<00:00,  4.08it/s]

retrying
conf is none
Write a critical analysis of the biological processes performed by this system of interacting proteins.

Base your analysis on prior knowledge available in your training data.  After completing your analysis, propose a brief and
detailed name for the most prominent biological process performed by the system.

After completing your analysis, please also assign a confidence score to the process name you selected.  This score should
follow the name in parentheses and range from 0.00 to 1.00. A score of 0.00 indicates the lowest confidence, while 1.00 reflects
the highest confidence. This score helps gauge how accurately the chosen name represents the functions and activities within
the system of interacting proteins. When determining your score, consider the proportion of genes in the protein system that
participate in the identified biological process. For instance, if you select "Ribosome biogenesis" as the process name but
only a few genes in the system contribute

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 801/801 [00:24<00:00, 33.04it/s]


In [5]:
df = pd.DataFrame(output)
df.to_csv("gsai_vs_llm2geneset_outputs.tsv", sep="\t", index=False)